In [98]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ast
import string
import re
from unidecode import unidecode
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet, stopwords
from collections import Counter
import nltk
vocabulary = nltk.FreqDist()
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
import _pickle as cPickle
import config
import spacy
import gc


In [69]:
# nlp = spacy.load("en_core_web_sm")
# spacy.load('en_core_web_sm')

In [70]:
interactions = pd.read_csv('./Data (remove before  commit)/RAW_interactions.csv').sort_values('rating').drop_duplicates('recipe_id') 
recipes_data = pd.read_csv('./Data (remove before  commit)/RAW_recipes.csv')

full_data = pd.merge(recipes_data, interactions, right_on=['recipe_id'], left_on=['id'])

In [71]:
# full_data.info()
# recipes.head(1)

# object = pd.read_pickle(r'/Users/scottsturtz/DSI/School_Work/Smash_Capstones/Capstone3/my_app/data/pandas_actual2.pkl')

In [72]:
# object.head(1)


In [73]:
data=full_data.copy()
recipes = pd.DataFrame(columns = ['_id','combined'  ,'direction_list' ,'ingred_list','item_name', 'stars', 'submitter_desc' ])
recipes[['_id','direction_list', 'ingred_list', 'item_name', 'stars', 'submitter_desc']] = data[['recipe_id', 'steps', 'ingredients', 'name', 'rating', 'description']].sort_values('recipe_id')
# recipes.drop(columns=['nutrition', 'tags', 'id', 'steps', 'ingredients', 'name', 'rating', 'description','contributor_id'	,'submitted','user_id','date',	'submitter_desc'	], inplace=True)






In [74]:
recipes.head(2)

,_id,combined,direction_list,ingred_list,item_name,stars,submitter_desc
126987,38,NaN,"['toss 2 cups berries with sugar', 'let stand ...","['blueberries', 'granulated sugar', 'vanilla y...",low fat berry blue frozen dessert,4,"this is yummy and low-fat, it always turns out..."
23695,39,NaN,['soak saffron in warm milk for 5 minutes and ...,"['saffron', 'milk', 'hot green chili peppers',...",biryani,3,"delhi, india"


In [75]:


# recipes['summary']= recipes['summary'].str.replace('\d+', '') #remove digits
recipes['summary']= recipes['direction_list'].str.replace('[^\w\s]', '')#remove punctuation

# Making recipe a string
recipes['instructions'] = recipes['direction_list'].apply(lambda x : " ".join(ast.literal_eval(x)))
# recipes['ingredients']=recipes['ingredients'].apply(lambda x : " ".join(ast.literal_eval(x)))


<ipython-input-75-0e7f36f3f483>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  recipes['summary']= recipes['direction_list'].str.replace('[^\w\s]', '')#remove punctuation


In [76]:
# # counting unique values
# n = len(pd.unique(interactions['recipe_id']))
  
# print("No.of.unique values :", 
#       n)

In [77]:
# for i in recipes['ingredients']:
#     i = i.split()
#     vocabulary.update(i)

# ingredient_bank = vocabulary

In [78]:
# for word, freq in ingredient_bank.most_common(200):
#     print(f'{word};{freq}')

In [79]:
def parser(ingreds):
    measurements = list(set(['teaspoon', 't', 'tsp.', 'tablespoon', 'T', 'tbl.', 'tb', 'tbsp.', 'fluid ounce', 'fl oz', 
    'gill', 'cup', 'c', 'pint', 'p', 'pt', 'fl pt', 'quart', 'q', 'qt', 'fl qt', 'gallon', 'g', 'gal', 'ml', 'milliliter',
     'millilitre', 'cc', 'mL', 'l', 'liter', 'litre', 'L', 'dl', 'deciliter', 'decilitre', 'dL', 'bulb', 'level', 'heaped', 
     'rounded', 'whole', 'pinch', 'medium', 'slice', 'pound', 'lb', '#', 'ounce', 'oz', 'mg', 'milligram', 'milligramme', 
     'g', 'gram', 'gramme', 'kg', 'kilogram', 'kilogramme', 'x', 'of', 'mm', 'millimetre', 'millimeter', 'cm', 'centimeter', 
     'centimetre', 'm', 'meter', 'metre', 'inch', 'in', 'milli', 'centi', 'deci', 'hecto', 'kilo']))
    # xtra_stop_words = ['fresh', 'oil', 'a', 'red', 'bunch', 'and', 'clove', 'or', 'leaf', 'chilli', 'large', 'extra', 'sprig',
    #  'ground', 'handful', 'free', 'small', 'pepper', 'virgin', 'range', 'from', 'dried', 'sustainable', 'black', 'peeled', 'higher',
    #   'welfare', 'seed', 'for', 'finely', 'freshly', 'sea', 'quality', 'white', 'ripe', 'few', 'piece', 'source', 'to', 'organic', 
    #   'flat', 'smoked', 'ginger', 'sliced', 'green', 'picked', 'the', 'stick', 'plain', 'plus', 'mixed', 'mint', 'bay', 'basil', 
    #   'your', 'cumin', 'optional', 'fennel', 'serve', 'mustard', 'unsalted', 'baby', 'paprika', 'fat', 'ask', 'natural', 'skin', 
    #   'roughly', 'into', 'such', 'cut', 'good', 'brown', 'grated', 'trimmed', 'oregano', 'powder', 'yellow', 'dusting', 'knob', 
    #   'frozen', 'on', 'deseeded', 'low', 'runny', 'balsamic', 'cooked', 'streaky', 'nutmeg', 'sage', 'rasher', 'zest', 'pin', 
    #   'groundnut', 'breadcrumb', 'turmeric', 'halved', 'grating', 'stalk', 'light', 'tinned', 'dry', 'soft', 'rocket', 'bone', 
    #   'colour', 'washed', 'skinless', 'leftover', 'splash', 'removed', 'dijon', 'thick', 'big', 'hot', 'drained', 'sized', 
    #   'chestnut', 'watercress', 'fishmonger', 'english', 'dill', 'caper', 'raw', 'worcestershire', 'flake', 'cider', 'cayenne', 
    #   'tbsp', 'leg', 'pine', 'wild', 'if', 'fine', 'herb', 'almond', 'shoulder', 'cube', 'dressing', 'with', 'chunk', 'spice', 
    #   'thumb', 'garam', 'new', 'little', 'punnet', 'peppercorn', 'shelled', 'saffron', 'other''chopped', 'salt', 'olive', 'taste', 
    #   'can', 'sauce', 'water', 'diced', 'package', 'italian', 'shredded', 'divided', 'parsley', 'vinegar', 'all', 'purpose', 
    #   'crushed', 'juice', 'more', 'coriander', 'bell', 'needed', 'thinly', 'boneless', 'half', 'thyme', 'cubed', 'cinnamon', 
    #   'cilantro', 'jar', 'seasoning', 'rosemary', 'extract', 'sweet', 'baking', 'beaten', 'heavy', 'seeded', 'tin', 'vanilla', 
    #   'uncooked', 'crumb', 'style', 'thin', 'nut', 'coarsely', 'spring', 'chili', 'cornstarch', 'strip', 'cardamom', 'rinsed', 
    #   'honey', 'cherry', 'root', 'quartered', 'head', 'softened', 'container', 'crumbled', 'frying', 'lean', 'cooking', 'roasted', 
    #   'warm', 'whipping', 'thawed', 'corn', 'pitted', 'sun', 'kosher', 'bite', 'toasted', 'lasagna', 'split', 'melted', 'degree', 
    #   'lengthwise', 'romano', 'packed', 'pod', 'anchovy', 'rom', 'prepared', 'juiced', 'fluid', 'floret', 'room', 'active', 
    #   'seasoned', 'mix', 'deveined', 'lightly', 'anise', 'thai', 'size', 'unsweetened', 'torn', 'wedge', 'sour', 'basmati', 
    #   'marinara', 'dark', 'temperature', 'garnish', 'bouillon', 'loaf', 'shell', 'reggiano', 'canola', 'parmigiano', 'round',
    #    'canned', 'ghee', 'crust', 'long', 'broken', 'ketchup', 'bulk', 'cleaned', 'condensed', 'sherry', 'provolone', 'cold', 
    #    'soda', 'cottage', 'spray', 'tamarind', 'pecorino', 'shortening', 'part', 'bottle', 'sodium', 'cocoa', 'grain', 'french', 
    #    'roast', 'stem', 'link', 'firm', 'asafoetida', 'mild', 'dash', 'boiling']

    if isinstance(ingreds, list):

        ingredients=ingreds
    else:
        ingredients = ast.literal_eval(ingreds)
        
    # ingredients = ','.join(ingredients)
    
    translator = str.maketrans("", "", string.punctuation)
    lemmatizer = WordNetLemmatizer()

    grocery_bag = []
    for i in ingredients:
        i.translate(translator)
        items = re.split(" |-",i)
        items = [word for word in items if word.isalpha()]
        items = [word.lower() for word in items]
        items = [unidecode(word) for word in items]
        # Lemmatize words 
        items = [lemmatizer.lemmatize(word) for word in items]
        # remove stop words
        stop_words = stopwords.words("english")
        items = [word for word in items if word not in stop_words]
        # remove measuring phrases
        items = [word for word in items if word not in measurements]
        # # remove misc words
        # items = [word for word in items if word not in xtra_stop_words]
        if items:
            # grocery_bag += items
           grocery_bag.append(" ".join(items))
    grocery_bag = " ".join(grocery_bag)
    return grocery_bag     
    


In [80]:

# market = parser(recipes['ingredients'])
#takes a long time 3536 s


In [81]:
recipes['combined'] = recipes['ingred_list'].apply(lambda x: parser(x)) 
#cleaning and organizing data
recipes.dropna(inplace=True)

# takes around 311 s

In [82]:
#cleaning and organizing data
recipes.head(3)

,_id,combined,direction_list,ingred_list,item_name,stars,submitter_desc,summary,instructions
126987,38,blueberry granulated sugar vanilla yogurt lemo...,"['toss 2 cups berries with sugar', 'let stand ...","['blueberries', 'granulated sugar', 'vanilla y...",low fat berry blue frozen dessert,4,"this is yummy and low-fat, it always turns out...",toss 2 cups berries with sugar let stand for 4...,toss 2 cups berries with sugar let stand for 4...
23695,39,saffron milk hot green chili pepper onion garl...,['soak saffron in warm milk for 5 minutes and ...,"['saffron', 'milk', 'hot green chili peppers',...",biryani,3,"delhi, india",soak saffron in warm milk for 5 minutes and pu...,soak saffron in warm milk for 5 minutes and pu...
22701,40,sugar rind fresh water fresh lemon juice ice c...,"['into a 1 quart jar with tight fitting lid , ...","['sugar', 'lemons, rind of', 'fresh water', 'f...",best lemonade,1,this is from one of my first good house keepi...,into a 1 quart jar with tight fitting lid put...,"into a 1 quart jar with tight fitting lid , pu..."


In [100]:
#feature extraction
tfidf = TfidfVectorizer(ngram_range=(2,3), sublinear_tf=True)
# ngram_range = (1,2)
# norm{‘l1’, ‘l2’}, default=’l2’
# sublinear_tf, default=False

# tfidf = TfidfVectorizer()
tfidf.fit(recipes['combined'])
recipe_feat = tfidf.transform(recipes['combined'])

'''
creating important pickle files for app to run
'''
with open('my_app/data/TFIDF_Model.pkl','wb') as f:
    pickle.dump(tfidf, f)
    

with open('my_app/data/Features.pkl', 'wb') as f:
    pickle.dump(recipe_feat, f)

with open('my_app/data/Pandas_Data.pkl','wb') as f:
    pickle.dump(recipes, f)

gc.disable()

file = open('my_app/data/TFIDF_Model2.pkl', 'wb')
cPickle.dump(tfidf, file, protocol=-1)
file.close()

file = open('my_app/data/Features2.pkl', 'wb')
cPickle.dump(recipe_feat, file, protocol=-1)
file.close()

file = open('my_app/data/Pandas_Data2.pkl', 'wb')
cPickle.dump(recipes, file, protocol=-1)
file.close()

gc.enable()

In [84]:
def reco_maker(top_n, scores):
    print(scores[:3])
    top = sorted(range(len(scores)), key= lambda i: scores[i], reverse=True)[:top_n]

    recos = pd.DataFrame(columns = ["recipe", "ingredients",'instructions', "score"])

    count= 0
    for i in top:
        recos.at[count, "recipe"] = format_title(recipes["item_name"][i])
        recos.at[count, "ingredients"] = recipes["combined"][i]
        recos.at[count, "instructions"] = pretty_grocery(recipes['direction_list'][i])
        # recos.at[count, "ingredients"] = pretty_grocery(recipes["combined"][i])
        # recos.at[count, "instructions"] = pretty_grocery(recipes['direction_list'][i])
        recos.at[count, "score"] = "{:.3f}".format(float(scores[i]))
        
        count += 1
    return recos



In [85]:
def format_title(title):
    title = unidecode(title)
    return title

def pretty_grocery(ingredient):
    
    if isinstance(ingredient, list):
        ingredients=ingredient
    else:
        ingredients = ast.literal_eval(ingredient)    #     ingredients = ast.parse(ingredient)
    # ingredients = ','.join(ingredient.split(' '))
    ingredients = ",".join(ingredients)
    ingredients = unidecode(ingredients)
    return ingredients

In [86]:
def recomatic(ingredients, top_n=5):

    with open('my_app/data/TFIDF_Model.pkl','rb') as f:
        tfidf = pickle.load(f)

    with open('my_app/data/Features.pkl', 'rb') as f:
        feats = pickle.load(f)
    feats = recipe_feat
    parsed = " ".join(ingredients)


    ingredients_tfidf = tfidf.transform([parsed])
    #cosine similarity  
    similarity= map(lambda x: cosine_similarity(ingredients_tfidf, x,), feats)
    scores = list(similarity)

    #filter reccomendations from sklearn.metrics.cluster import fowlkes_mallows_score

    reccomendations = reco_maker(top_n, scores)

    return reccomendations


if __name__ == "__main__":
    # test ingredients
    test_ingredients = ['steak', 'pasta', 'tomatoes']
    recs = recomatic(test_ingredients)
    print(recs)
    # takes around 500 s

[array([[0.]]), array([[0.]]), array([[0.]])]
                                           recipe  \
0      warm grilled salmon  mushroom   leek salad   
1  microwave bread and butter pickles  sugar free   
2      arriba   baked winter squash mexican style   
3                a bit different  breakfast pizza   
4                       all in the kitchen  chili   

                                         ingredients  \
0  new potato extra virgin olive oil salt pepper ...   
1  cucumber sweet onion splenda granular salt cel...   
2  winter squash mexican seasoning mixed spice ho...   
3  prepared pizza crust sausage patty egg milk sa...   
4  ground beef yellow onion diced tomato tomato p...   

                                        instructions  score  
0  cook potatoes in a large pot of boiling salted...  0.358  
1  in a large microwave-safe bowl , combine all i...  0.260  
2  make a choice and proceed with recipe,dependin...  0.000  
3  preheat oven to 425 degrees f,press dough into.

In [87]:
'''
using steak pasta tomatos
Adding ngram range
 recipe  \
0                             bagels   prosciutto   
1              easy devils food chocolate muffins   
2  microwave bread and butter pickles  sugar free   


tfidf = TfidfVectorizer(ngram_range=(2,2))
0      warm grilled salmon  mushroom   leek salad   
1            best fresh margaritas by the pitcher   
2  microwave bread and butter pickles  sugar free   
3                    indian sprouted lentil salad   
4                              risotto style soup  

tfidf = TfidfVectorizer(ngram_range=(2,2), norm = 'l1')
                                          recipe  \
0      warm grilled salmon  mushroom   leek salad   
1            best fresh margaritas by the pitcher   
2  microwave bread and butter pickles  sugar free   
3                    indian sprouted lentil salad   
4                              risotto style soup   

tfidf = TfidfVectorizer(ngram_range=(2,2), norm = 'l1', sublinear_tf=True)
                                          recipe  \
0      warm grilled salmon  mushroom   leek salad   
1            best fresh margaritas by the pitcher   
2  microwave bread and butter pickles  sugar free   
3                    indian sprouted lentil salad   
4                              risotto style soup   
'''

"\nusing steak pasta tomatos\nAdding ngram range\n recipe  0                             bagels   prosciutto   \n1              easy devils food chocolate muffins   \n2  microwave bread and butter pickles  sugar free   \n\n\ntfidf = TfidfVectorizer(ngram_range=(2,2))\n0      warm grilled salmon  mushroom   leek salad   \n1            best fresh margaritas by the pitcher   \n2  microwave bread and butter pickles  sugar free   \n3                    indian sprouted lentil salad   \n4                              risotto style soup  \n\ntfidf = TfidfVectorizer(ngram_range=(2,2), norm = 'l1')\n                                          recipe  0      warm grilled salmon  mushroom   leek salad   \n1            best fresh margaritas by the pitcher   \n2  microwave bread and butter pickles  sugar free   \n3                    indian sprouted lentil salad   \n4                              risotto style soup   \n\ntfidf = TfidfVectorizer(ngram_range=(2,2), norm = 'l1', sublinear_tf=True)\n     

In [88]:
type(recs)
recs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5 entries, 0 to 4
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   recipe        5 non-null      object
 1   ingredients   5 non-null      object
 2   instructions  5 non-null      object
 3   score         5 non-null      object
dtypes: object(4)
memory usage: 372.0+ bytes


In [89]:
t1 = recs.loc[0][0]
ingred1 = recs.loc[0][1]
instructions1 = recs.loc[0][2]
score1 = recs.loc[0][3]

t2 = recs.loc[1][0]
ingred2 = recs.loc[1][1]
instructions2 = recs.loc[0][2]
score2 = recs.loc[1][3]

t3 = recs.loc[2][0]
ingred3 = recs.loc[2][1]
instructions3 = recs.loc[2][2]
score3 = recs.loc[2][3]

t4 = recs.loc[3][0]
ingred4 = recs.loc[3][1]
instructions4 = recs.loc[3][2]
score4 = recs.loc[3][3]



In [90]:
print(instructions4)

preheat oven to 425 degrees f,press dough into the bottom and sides of a 12 inch pizza pan,bake for 5 minutes until set but not browned,cut sausage into small pieces,whisk eggs and milk in a bowl until frothy,spoon sausage over baked crust and sprinkle with cheese,pour egg mixture slowly over sausage and cheese,s& p to taste,bake 15-20 minutes or until eggs are set and crust is brown


In [91]:
    # with open('my_app/data/Features.pkl', 'rb') as f:
    #     feats = pickle.load(f)

In [92]:
# full= pd.merge(recipes, ratings, right_on=['recipe_id'], left_on=['id']) 

In [93]:
import pandas as pd
import pickle as cPickle
# pickle.load('my_app/data/model_actual.pkl')
# t = pd.read_pickle(r'my_app/data/model_actual.pkl')

# objects = []
# with (open('my_app/data/model_actual.pkl', "rb")) as openfile:
#     while True:
#         try:
#             objects.append(pickle.load(openfile))
#         except EOFError:
#             break
# while True:
#     try:
#         objects.append(pickle.load(pickle_file))
#     except EOFError:
#         break
# pickle_file.close()
# p = load('my_app/data/model_actual.pk', 'rb')




In [94]:
update=full_data.copy()


update.head(2)
# update.sortby(id)

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,user_id,recipe_id,date,rating,review
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7,4470,137739,2006-02-18,5,I used an acorn squash and recipe#137681 Swee...
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6,28603,31490,2002-08-19,0,"Have not tried this, but it sounds delicious. ..."


In [95]:
update['stars'] = update.rating.mean().groupby('recipe_id')

update.head()

AttributeError: 'float' object has no attribute 'groupby'

In [ ]:
# filename = 'pandas_data'
# outfile = open(filename, 'wb')

# pickle.dump(update, outfile)
# outfile.close